# Credit Analytics - Final Project Code
By: Dennis Goldenberg, Triet Vo, Kezia 

In [1]:
import numpy as np
from scipy.stats import norm, binom

## 1. Comparing Binomial Expansion Model to Gaussian Copula Model

In [5]:
num_assets = 1000
Notional = 100000000
Notional/num_assets

100000.0

## 2. Comparing Gaussian Copula to Fudged Period by Period Copula